In [585]:
#Importing all the packages and the CSV file
import pandas as pd
import numpy as np
from pandasql import sqldf
customer_table = pd.read_csv('Customer table.csv')

In [586]:
#Removing the timestamp values from the switch columns
customer_table['switch_start'] = customer_table['switch_start'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d'))
customer_table['switch_end'] = customer_table['switch_end'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d'))

#CSV Preview
print(customer_table)

      id  customer_id  switches   value switch_start  switch_end
0      1         1111         5    99,9   2022-01-10  2022-12-31
1      2         1111         5  101,99   2023-01-01  2023-01-31
2      3         1111         5  103,98   2023-02-01  2023-02-28
3      4         1111         5  105,97   2023-03-01  2023-03-31
4      5         1111         5  107,96   2023-04-01  2023-05-01
..   ...          ...       ...     ...          ...         ...
145  146         1239         1    99,9   2023-01-01  2024-01-01
146  147         1240         1    99,9   2023-02-01  2024-02-01
147  148         1241         1    99,9   2023-03-01  2024-02-29
148  149         1242         1    99,9   2023-01-01  2024-01-01
149  150         1243         1    99,9   2023-02-01  2024-02-01

[150 rows x 6 columns]


In [421]:
#Importing datetime to generate a dataframe with all dates from the beginning until today
from datetime import datetime

# start and end date
now = datetime.now()
start_date = datetime.strptime("2022-01-10", "%Y-%m-%d")
end_date = now.strftime("%Y-%m-%d")

#Preview
date_list = pd.date_range(start_date, end_date, freq='D')
print(f"Creating list of dates starting from {start_date} to {end_date}")
print(date_list)

Creating list of dates starting from 2022-01-10 00:00:00 to 2023-05-03
DatetimeIndex(['2022-01-10', '2022-01-11', '2022-01-12', '2022-01-13',
               '2022-01-14', '2022-01-15', '2022-01-16', '2022-01-17',
               '2022-01-18', '2022-01-19',
               ...
               '2023-04-24', '2023-04-25', '2023-04-26', '2023-04-27',
               '2023-04-28', '2023-04-29', '2023-04-30', '2023-05-01',
               '2023-05-02', '2023-05-03'],
              dtype='datetime64[ns]', length=479, freq='D')


In [550]:
from pandas.tseries.offsets import MonthEnd

#Creating the date dataframe
dates_df = pd.DataFrame({'Datetime': date_list})
dates_df['EOM'] = pd.to_datetime(dates_df['Datetime'], format='%Y-%m-%d') + MonthEnd(0)

#Transforming all the dates in yyyy-mm-dd
dates_df['Datetime'] = dates_df['Datetime'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d'))
dates_df['EOM'] = dates_df['EOM'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d'))


#Overview of dates
print(dates_df)

       Datetime         EOM
0    2022-01-10  2022-01-31
1    2022-01-11  2022-01-31
2    2022-01-12  2022-01-31
3    2022-01-13  2022-01-31
4    2022-01-14  2022-01-31
..          ...         ...
474  2023-04-29  2023-04-30
475  2023-04-30  2023-04-30
476  2023-05-01  2023-05-31
477  2023-05-02  2023-05-31
478  2023-05-03  2023-05-31

[479 rows x 2 columns]


In [551]:
#Transforming all the dates in yyyy-mm-dd from the CSV file
customer_table['switch_start'] = df['switch_start'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d'))
customer_table['switch_end'] = df['switch_end'].apply(lambda x: pd.Timestamp(x).strftime('%Y-%m-%d'))

In [587]:
#Testing SQL environment 
pysqldf = lambda q: sqldf(q)
pysqldf("SELECT * FROM customer_table")

,id,customer_id,switches,value,switch_start,switch_end
0,1,1111,5,"99,9",2022-01-10,2022-12-31
1,2,1111,5,"101,99",2023-01-01,2023-01-31
2,3,1111,5,"103,98",2023-02-01,2023-02-28
3,4,1111,5,"105,97",2023-03-01,2023-03-31
4,5,1111,5,"107,96",2023-04-01,2023-05-01
...,...,...,...,...,...,...
145,146,1239,1,"99,9",2023-01-01,2024-01-01
146,147,1240,1,"99,9",2023-02-01,2024-02-01
147,148,1241,1,"99,9",2023-03-01,2024-02-29
148,149,1242,1,"99,9",2023-01-01,2024-01-01


In [588]:
# Selecting one customer to test the join
customer_1111 = pysqldf("""
           select 
           DISTINCT
           dd.EOM, 
            ct.*
           from dates_df dd
               left join customer_table ct
                   on dd.EOM >= ct.switch_start 
                       and dd.EOM <= switch_end
           where customer_id = 1111
           order by 1 asc
        """)

print(customer_1111)

           EOM  id  customer_id  switches   value switch_start  switch_end
0   2022-01-31   1         1111         5    99,9   2022-01-10  2022-12-31
1   2022-02-28   1         1111         5    99,9   2022-01-10  2022-12-31
2   2022-03-31   1         1111         5    99,9   2022-01-10  2022-12-31
3   2022-04-30   1         1111         5    99,9   2022-01-10  2022-12-31
4   2022-05-31   1         1111         5    99,9   2022-01-10  2022-12-31
5   2022-06-30   1         1111         5    99,9   2022-01-10  2022-12-31
6   2022-07-31   1         1111         5    99,9   2022-01-10  2022-12-31
7   2022-08-31   1         1111         5    99,9   2022-01-10  2022-12-31
8   2022-09-30   1         1111         5    99,9   2022-01-10  2022-12-31
9   2022-10-31   1         1111         5    99,9   2022-01-10  2022-12-31
10  2022-11-30   1         1111         5    99,9   2022-01-10  2022-12-31
11  2022-12-31   1         1111         5    99,9   2022-01-10  2022-12-31
12  2023-01-31   2       

In [590]:
#Checking all customer ids and the number of switches, rows and min/max dates
customer_check = pysqldf("""
            select 
                distinct customer_id, 
                switches,
                count(distinct value) as distinct_switch_value,
                count (*) as row_count,
                min(EOM) as min_date, 
                max(EOM) as max_date
            from dates_df dd
                left join customer_table ct
                    on dd.EOM >= ct.switch_start 
                    and dd.EOM <= switch_end
            group by 1,2 
        """)

print(customer_check)

     customer_id  switches  distinct_switch_value  row_count    min_date  \
0           1111         5                      5        476  2022-01-31   
1           1112         2                      2        123  2023-01-31   
2           1113         3                      3        123  2023-01-31   
3           1114         1                      1        154  2022-12-31   
4           1115         1                      1        154  2022-12-31   
..           ...       ...                    ...        ...         ...   
128         1239         1                      1        123  2023-01-31   
129         1240         1                      1         92  2023-02-28   
130         1241         1                      1         64  2023-03-31   
131         1242         1                      1        123  2023-01-31   
132         1243         1                      1         92  2023-02-28   

       max_date  
0    2023-04-30  
1    2023-05-31  
2    2023-05-31  
3    2023-05-31

In [591]:
#Final output
customer_final = pysqldf("""
           select 
           DISTINCT
            dd.EOM, 
            ct.*
           from dates_df dd
               left join customer_table ct
                   on dd.EOM >= ct.switch_start 
                       and dd.EOM <= switch_end
           order by 1 asc
        """)

print(customer_final)

            EOM   id  customer_id  switches value switch_start  switch_end
0    2022-01-31    1         1111         5  99,9   2022-01-10  2022-12-31
1    2022-01-31   18         1120         5  99,9   2022-01-10  2022-12-31
2    2022-01-31   69         1166         4   101   2022-01-10  2022-12-31
3    2022-02-28    1         1111         5  99,9   2022-01-10  2022-12-31
4    2022-02-28   18         1120         5  99,9   2022-01-10  2022-12-31
..          ...  ...          ...       ...   ...          ...         ...
687  2023-05-31  146         1239         1  99,9   2023-01-01  2024-01-01
688  2023-05-31  147         1240         1  99,9   2023-02-01  2024-02-01
689  2023-05-31  148         1241         1  99,9   2023-03-01  2024-02-29
690  2023-05-31  149         1242         1  99,9   2023-01-01  2024-01-01
691  2023-05-31  150         1243         1  99,9   2023-02-01  2024-02-01

[692 rows x 7 columns]


In [592]:
#Exporting CSV
customer_final.to_csv('customer_final.csv', index=False)

In [559]:
#import library and data
import cutecharts.charts as ctc

In [594]:
#Output for acquired cohort analysis
acquired_cohort = pysqldf("""
            WITH 
            customer_table_idx AS 
            (
            SELECT 
                switch_start,
                switch_end,
                customer_id,
                value as switch_value,
                row_number() over (partition by customer_id order by switch_start asc) as first_idx,
                row_number() over (partition by customer_id order by switch_start desc) as last_idx
            FROM customer_table
            )
           select 
           DISTINCT 
            strftime ('%Y/%m',dd.EOM) AS EOM,
            sum(switch_value) as value,
            count(distinct customer_id) as customers
           from dates_df dd
               left join customer_table_idx ct
                   on dd.EOM >= ct.switch_start 
                       and dd.EOM <= switch_end
           where first_idx = 1
          group by 1 order by 1 asc
        """)

print(acquired_cohort)

        EOM     value  customers
0   2022/01    6578.0          3
1   2022/02    8372.0          3
2   2022/03    9269.0          3
3   2022/04    8970.0          3
4   2022/05    9269.0          3
5   2022/06    8970.0          3
6   2022/07    9269.0          3
7   2022/08    9269.0          3
8   2022/09    8970.0          3
9   2022/10   24614.0          8
10  2022/11   41640.0         14
11  2022/12  211823.0         69
12  2023/01  264182.0         86
13  2023/02  266112.0         96
14  2023/03  352935.0        115
15  2023/04  356400.0        120
16  2023/05   37125.0        125


In [603]:
import cutecharts.globals as ctcg


chart = ctc.Bar('Acquired Customers at EOM',width='1200px',height='800px')
chart.set_options(
 labels=list(acquired_cohort['EOM']),
 y_label='Number of Acquired Customers' 
 )
chart.add_series('Acquired Customers' ,list(acquired_cohort['customers']))
chart.render_notebook()